# Training Workbook for Supercomputer Environment

## Setup

In [1]:
## Setup and configs
# imports
import os
import time
import matplotlib
import matplotlib.pyplot as plt
# global variables
global notebook
global axisym,set_cart,axisym,REF_1,REF_2,REF_3,set_cart,D,print_fieldlines
global lowres1,lowres2,lowres3, RAD_M1, RESISTIVE, export_raytracing_GRTRANS, export_raytracing_RAZIEH,r1,r2,r3
global r_min, r_max, theta_min, theta_max, phi_min,phi_max, do_griddata, do_box, check_files, kerr_schild

notebook = 1

# total data is shape (10000, 224, 48, 96)
harm_directory = os.environ['HOME']+f'/bh/harm2d'
os.chdir(harm_directory)

print(f'Running setup scripts...')
start_time = time.time()
%run -i setup.py build_ext --inplace
%run -i pp.py build_ext --inplace
print(f"Execution time: {time.time() - start_time}")

# set params
lowres1 = 1 # 
lowres2 = 1 # 
lowres3 = 1 # 
r_min, r_max = 1.0, 100.0
theta_min, theta_max = 0.0, 9
phi_min, phi_max = -1, 9
do_box=0
set_cart=0
set_mpi(0)
axisym=1
print_fieldlines=0
export_raytracing_GRTRANS=0
export_raytracing_RAZIEH=0
kerr_schild=0
DISK_THICKNESS=0.03
check_files=1
notebook=1
interpolate_var=0
AMR = 0 # get all data in grid

print('Imports and setup done.')
%matplotlib inline

Running setup scripts...
Starting single GPU training
--- Training script running! ---


RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [Sequential: 1, Conv3d: 2, MaxPool3d: 2, ReLU: 2, Conv3d: 2, MaxPool3d: 2, ReLU: 2, Flatten: 2, Linear: 2, Sigmoid: 2, Flatten: 2]

Execution time: 6.3288185596466064
Imports and setup done.


## Training

In [4]:
# imports
import numpy as np
import torch
from tqdm import tqdm

# training utilities
from utils.sc_utils import custom_batcher, tensorize_globals
from models.cnn.cnn import CNN_3D

# path to dumps
dumps_path = '/pscratch/sd/l/lalakos/ml_data_rc300/reduced'
os.chdir(dumps_path)
# number of data points
num_dumps = 11 - 1
# batch size
batch_size = 2
# number of epochs
num_epochs = 2

# access device, cuda device if accessible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# set model
model = CNN_3D().to(device)
# set loss
optim = torch.optim.Adam(params=model.parameters())
loss_fn = torch.nn.MSELoss()

# get indexes for training data
train_indexes, validation_indexes = custom_batcher(
    batch_size=batch_size,
    num_dumps=num_dumps,
    split = 0.8,
    seed=1
)

num_train_batches = len(train_indexes)//batch_size
num_valid_batches = len(validation_indexes)//batch_size

best_validation = float('inf')

rgdump_griddata(dumps_path)

for epoch in range(num_epochs):
    ## Training
    model.train()
    epoch_train_loss = []

    # shuffle training indexes
    np.random.shuffle(train_indexes)

    # list of average train/validation losses after each epoch
    train_losses, valid_losses = [], []

    prog_bar = tqdm(enumerate(train_indexes.reshape(-1, batch_size)), total=num_train_batches)
    for batch_num, batch_indexes in prog_bar:
        ## fetch and tensorize data
        # NOTE everything is a global variable so it has to be this way. im sorry
        batch_data, label_data = [], []
        # batch_idx is the dump number
        for batch_idx in batch_indexes:

            # at every batch of size batch_size, we need to read in 2 * batch_size dumps
            
            ## get data frame
            # get data into global context NOTE this is really slow
            rblock_new(batch_idx)
            rpar_new(batch_idx)
            rdump_griddata(dumps_path, batch_idx)
            # format data as tensor
            data_tensor = tensorize_globals(rho=rho, ug=ug, uu=uu, B=B)
            # add to batch
            batch_data.append(data_tensor)

            ## get label frame
            # get data into global context
            rblock_new(batch_idx+1)
            rpar_new(batch_idx+1)
            rgdump_griddata(dumps_path)
            rdump_griddata(dumps_path, batch_idx+1)
            # format data as tensor
            data_tensor = tensorize_globals(rho=rho, ug=ug, uu=uu, B=B)
            # add to batch
            label_data.append(data_tensor)

        # final tensorize
        batch_data = torch.cat(batch_data, dim=0).to(device)
        label_data = torch.cat(label_data, dim=0).to(device)

        ## train model
        # make prediction
        pred = model.forward(batch_data)
        # compute loss
        loss_value = loss_fn(pred, label_data)
        epoch_train_loss.append(loss_value)
        # backprop
        loss_value.backward()
        # update paramts
        optim.step()

        prog_bar.set_description(f'Train batch {batch_num+1} completed with loss {loss_value.item():.4f}')

    # training loss tracking
    avg_loss_after_epoch = sum(epoch_train_loss)/len(epoch_train_loss)
    train_losses.append(avg_loss_after_epoch)
    print(f"Train loss value: {avg_loss_after_epoch}")


    ## Validation
    model.eval()
    epoch_valid_loss = []

    prog_bar = tqdm(enumerate(validation_indexes.reshape(-1, batch_size)), total=num_valid_batches)
    for batch_num, batch_indexes in prog_bar:
        ## fetch and tensorize data
        # NOTE everything is a global variable so it has to be this way. im sorry
        batch_data, label_data = [], []
        # batch_idx is the dump number
        for batch_idx in batch_indexes:
            ## get data frame
            # get data into global context
            rblock_new(batch_idx)
            rpar_new(batch_idx)
            rgdump_griddata(dumps_path)
            rdump_griddata(dumps_path, batch_idx)
            # format data as tensor
            data_tensor = tensorize_globals(rho=rho, ug=ug, uu=uu, B=B)
            # add to batch
            batch_data.append(data_tensor)

            ## get label frame
            # get data into global context
            rblock_new(batch_idx+1)
            rpar_new(batch_idx+1)
            rgdump_griddata(dumps_path)
            rdump_griddata(dumps_path, batch_idx+1)
            # format data as tensor
            data_tensor = tensorize_globals(rho=rho, ug=ug, uu=uu, B=B)
            # add to batch
            label_data.append(data_tensor)

        # final tensorize
        batch_data = torch.cat(batch_data, dim=0).to(device)
        label_data = torch.cat(label_data, dim=0).to(device)

        # make prediction
        pred = model.forward(batch_data)

        # compute loss
        loss_value = loss_fn(pred, label_data)
        epoch_valid_loss.append(loss_value)
        
        prog_bar.set_description(f'Validation batch {batch_num+1} completed with loss {loss_value.item():.4f}.')
        
    avg_vloss_after_epoch = sum(epoch_train_loss)/len(epoch_train_loss)
    valid_losses.append(avg_vloss_after_epoch)
    print(f"Valid loss value: {avg_loss_after_epoch}")

    # checkpointing
    if avg_vloss_after_epoch < best_validation:
        best_validation = avg_vloss_after_epoch
        save_path = os.environ['HOME'] + '/bh/' + model.save_path
        model.save(save_path=save_path)

# plot learning
plt.plot([i for i in range(len(train_losses))], [loss.item() for loss in train_losses], label='Train Loss')
# plt.plot([i for i in range(len(train_losses))], [avg_baseline_loss for _ in range(len(train_losses))], label='Predicting Avg Loss', linestyle='dashed')
plt.plot([i for i in range(len(valid_losses))], [loss.item() for loss in valid_losses], label='Validation Loss')
plt.title(f'Training and Validation Curve')
plt.xlabel(f'Number of Batches')
plt.ylabel(f'Loss (MSE)')
plt.legend()
plt.show()


  0%|          | 0/4 [00:05<?, ?it/s]


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 28 but got size 224 for tensor number 1 in the list.

## Visualization

In [1]:
# load in model
import os
import torch
from models.cnn.cnn import CNN_3D

# access device, cuda device if accessible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_path = os.environ['HOME'] + '/bh/models/cnn/saves/3dcnn_v0.0.0.pth'
model = CNN_3D()
model.load_state_dict(torch.load(f=model_path))
model = model.to(device)
print(f'Model loaded.')

Model loaded.


In [2]:
from utils.anim import make_prediciton_frames

os.chdir(os.environ['HOME'] + '/bh/')

# access device, cuda device if accessible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

save_path = os.environ['HOME']+'/bh/data.pkl'
data = torch.load(f=save_path)
# print(data.shape)

predictions, latents = make_prediciton_frames(
    net=model,
    first_frame=data[0].unsqueeze(0), 
    make_latents=False,
    num_frames=10,
    device=device
)

# animate
# animate_preds(
#     predictions = predictions, 
#     save_path = './movies/preds_movie.gif',
#     cb = True
# )

NameError: name 'animate_preds' is not defined

In [3]:
print(len(predictions))

10
